In [1]:
import pandas as pd
import numpy as np

In [63]:
df_bls = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.xlsx')
df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv')
df_rm_FY17 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2017_50_rev.xlsx')
df_rm_FY18 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2018_50_County_rev.xlsx')
df_rm_FY19 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2019_50_County_rev.xlsx')
df_rm_FY20 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2020_50_County_rev.xlsx')
df_rm_FY21 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2021_50_County.xlsx')
df_rm_FY22 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2022_FMR_50_county_rev.xlsx')
df_rm_FY23 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2023_FMR_50_county.xlsx')
df_me = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Medicare_Monthly_Enrollment_September_2022.csv')

df_cs = pd.read_csv('..\Data\data_zip\census_starter.csv')
df_test = pd.read_csv(r'..\Data\data_zip\test.csv')
df_train = pd.read_csv(r'..\Data\data_zip\train.csv')

In [64]:
df_train['first_day_of_month'] = pd.to_datetime(df_train['first_day_of_month'])
df_train['year'] = df_train['first_day_of_month'].dt.year

In [65]:
df_bls2 = df_bls[df_bls['period'] != 'M13']
df_bls2_2 = pd.pivot_table(df_bls2, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2_2['first_day_of_month'] = pd.to_datetime(df_bls2_2['first_day_of_month'])
df_bls2_2['first_day_of_month_lag_12'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=12)
df_bls2_2 = df_bls2_2.drop(columns=['Labor force'])
df_train_comb = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_12']
                               , suffixes=['_orig', '_bls'])
df_train_comb

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,Unemployment,Unemployment rate,first_day_of_month_lag_12
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,1012.0,3.8,2019-08-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,911.0,3.5,2019-09-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,912.0,3.4,2019-10-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,846.0,3.2,2019-11-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,879.0,3.3,2019-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,NaN,NaN,NaT
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,NaN,NaN,NaT
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,NaN,NaN,NaT
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,NaN,NaN,NaT


In [66]:
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
df_cs2.sort_values(['cfips']).reset_index(drop=True)
df_cs2 = pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_cs2['year_lag'] = (pd.to_datetime(df_cs2['Year']) + pd.DateOffset(years=2)).dt.year
df_cs2 = df_cs2.drop(columns=['median_hh_inc'])
df_train_comb = df_train_comb.merge(df_cs2, how='left', left_on=['cfips', 'year'], right_on=['cfips','year_lag'], 
                              suffixes=['_orig', '_cps'])
df_train_comb

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,Unemployment,Unemployment rate,first_day_of_month_lag_12,Year,pct_bb,pct_college,pct_foreign_born,pct_it_workers,year_lag
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,1012.0,3.8,2019-08-01,2017,76.6,14.5,2.1,1.3,2019
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,911.0,3.5,2019-09-01,2017,76.6,14.5,2.1,1.3,2019
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,912.0,3.4,2019-10-01,2017,76.6,14.5,2.1,1.3,2019
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,846.0,3.2,2019-11-01,2017,76.6,14.5,2.1,1.3,2019
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,879.0,3.3,2019-12-01,2017,76.6,14.5,2.1,1.3,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,NaN,NaN,NaT,2020,79.7,12.7,2.3,0.0,2022
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,NaN,NaN,NaT,2020,79.7,12.7,2.3,0.0,2022
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,NaN,NaN,NaT,2020,79.7,12.7,2.3,0.0,2022
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,NaN,NaN,NaT,2020,79.7,12.7,2.3,0.0,2022


In [67]:
df_hc['date'] = pd.to_datetime(df_hc['month_date_yyyymm'].apply(str).str[0:4]+'-'\
                               +df_hc['month_date_yyyymm'].apply(str).str[4:6]+'-01')
keep = ['date', 'county_fips', 'active_listing_count', 'active_listing_count_yy', 'median_days_on_market', 
        'new_listing_count', 'price_reduced_count']
df_hc_vars = df_hc[keep]
df_hc_vars['date_lag_12'] = df_hc_vars['date'] + pd.DateOffset(months=12)
df_train_comb = df_train_comb.merge(df_hc_vars, how='left', left_on=['cfips', 'first_day_of_month_orig'], \
                                    right_on=['county_fips','date_lag_12'], suffixes=['_orig', '_hc'])
df_train_comb

C:\Users\jacks\AppData\Local\Temp/ipykernel_15928/571208765.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hc_vars['date_lag_12'] = df_hc_vars['date'] + pd.DateOffset(months=12)


,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,pct_it_workers,year_lag,date,county_fips,active_listing_count,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count,date_lag_12
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,1.3,2019,2018-08-01,1001.0,226.0,0.0416,76.0,66.0,56.0,2019-08-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,...,1.3,2019,2018-09-01,1001.0,221.0,-0.0391,76.0,88.0,56.0,2019-09-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,...,1.3,2019,2018-10-01,1001.0,202.0,-0.1534,76.0,60.0,54.0,2019-10-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,...,1.3,2019,2018-11-01,1001.0,201.0,-0.1206,74.0,62.0,32.0,2019-11-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,...,1.3,2019,2018-12-01,1001.0,200.0,-0.0566,76.0,80.0,24.0,2019-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,...,0.0,2022,2021-06-01,56045.0,45.0,-0.2458,63.0,12.0,0.0,2022-06-01
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,...,0.0,2022,2021-07-01,56045.0,50.0,-0.2063,58.0,24.0,12.0,2022-07-01
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,0.0,2022,2021-08-01,56045.0,62.0,-0.0238,62.0,12.0,8.0,2022-08-01
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,0.0,2022,2021-09-01,56045.0,55.0,-0.0265,75.0,10.0,10.0,2022-09-01


In [68]:
df_me_yearly = df_me[(df_me['MONTH'] == 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)

df_me_monthly = df_me[(df_me['MONTH'] != 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)

df_me_monthly['date'] = pd.to_datetime(df_me_monthly['YEAR'].astype(str)  + df_me_monthly['MONTH'], format='%Y%B')


C:\Users\jacks\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\jacks\AppData\Local\Temp/ipykernel_15928/814155793.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_monthly['date'] = pd.to_datetime(df_me_monthly['YEAR'].astype(str)  + df_me_monthly['MONTH'], format='%Y%B')


In [69]:
date_df = pd.DataFrame(pd.date_range(start='2017-01-01', end='2021-08-01'), columns=['date'])

date_df['year'] = date_df.date.dt.year
date_df['day'] = date_df.date.dt.day
date_df = date_df[date_df.day == 1]

df_me_yearly_month = df_me_yearly.merge(date_df, how="cross")
df_me_yearly_month = df_me_yearly_month.drop(['year', 'day'], axis=1)
df_me_yearly_month

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
0,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-01-01
1,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-02-01
2,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-03-01
3,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-04-01
4,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917163,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-04-01
917164,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-05-01
917165,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-06-01
917166,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-07-01


In [70]:
df_me_final = pd.concat([df_me_monthly, df_me_yearly_month])
df_me_final = df_me_final.replace({'*': np.nan})
df_me_final['date_lag_12'] = df_me_final['date'] + pd.DateOffset(months=12)
keep = ['BENE_FIPS_CD','date_lag_12', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
df_me_final_keep = df_me_final[keep]
me_values = ['TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
for i in me_values:
    df_me_final_keep[i] = df_me_final_keep[i].astype(float)

df_train_comb = df_train_comb.merge(df_me_final_keep, how='left', left_on=['cfips', 'first_day_of_month_orig'], right_on=['BENE_FIPS_CD','date_lag_12'], 
                              suffixes=['_orig', '_me'])
df_train_comb

C:\Users\jacks\AppData\Local\Temp/ipykernel_15928/1918199496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_final_keep[i] = df_me_final_keep[i].astype(float)


,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,new_listing_count,price_reduced_count,date_lag_12_orig,BENE_FIPS_CD,date_lag_12_me,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,66.0,56.0,2019-08-01,1001,2019-08-01,10510.0,8046.0,2463.0,9810.0,7009.0
1,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,66.0,56.0,2019-08-01,1001,2019-08-01,10645.0,8241.0,2403.0,9892.0,7088.0
2,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,66.0,56.0,2019-08-01,1001,2019-08-01,11059.0,8596.0,2463.0,10249.0,7414.0
3,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,66.0,56.0,2019-08-01,1001,2019-08-01,11251.0,8840.0,2411.0,10425.0,7612.0
4,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,66.0,56.0,2019-08-01,1001,2019-08-01,11396.0,9031.0,2365.0,10555.0,7838.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586166,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,12.0,8.0,2022-08-01,56045,2022-08-01,1629.0,1454.0,175.0,1510.0,1067.0
586167,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,12.0,8.0,2022-08-01,56045,2022-08-01,1664.0,1496.0,168.0,1538.0,1100.0
586168,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,12.0,8.0,2022-08-01,56045,2022-08-01,1713.0,1555.0,158.0,1588.0,1133.0
586169,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,10.0,10.0,2022-09-01,56045,2022-09-01,1716.0,1555.0,161.0,1591.0,1135.0


In [71]:
df_rm_FY17['fips'] = df_rm_FY17['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY17['year'] = 2017
df_rm_FY17 = df_rm_FY17.rename(columns={'Rent50_0': 'rent50_0', 'Rent50_1': 'rent50_1', 'Rent50_2': 'rent50_2', 'Rent50_3': 'rent50_3', 'Rent50_4': 'rent50_4'})
df_rm_FY17_keep = df_rm_FY17[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY18['fips'] = df_rm_FY18['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY18['year'] = 2018
df_rm_FY18_keep = df_rm_FY18[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY19['fips'] = df_rm_FY19['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY19['year'] = 2019
df_rm_FY19_keep = df_rm_FY19[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY20['fips'] = df_rm_FY20['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY20['year'] = 2020
df_rm_FY20_keep = df_rm_FY20[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY21['fips'] = df_rm_FY21['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY21['year'] = 2021
df_rm_FY21_keep = df_rm_FY21[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY22['fips'] = df_rm_FY22['state_code'].astype(str) + df_rm_FY22['county_code'].astype(str).str.zfill(3)
df_rm_FY22['fips'] = df_rm_FY22['fips'].astype(int)
df_rm_FY22['year'] = 2022
df_rm_FY22 = df_rm_FY22.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY22_keep = df_rm_FY22[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY23['fips'] = df_rm_FY23['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY23['year'] = 2023
df_rm_FY23 = df_rm_FY23.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY23_keep = df_rm_FY23[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm = pd.concat([df_rm_FY17_keep, df_rm_FY18_keep, df_rm_FY19_keep, df_rm_FY20_keep, df_rm_FY21_keep, df_rm_FY22_keep, df_rm_FY23_keep])

df_train_comb = df_train.merge(df_train_comb, how='left', left_on=['cfips', 'year'], right_on=['fips','year'], 
                              suffixes=['_orig', 'rm'])

df_train_comb

KeyError: 'fips'